# Object Detection for Face Detection Project

This notebook demonstrates the use of pre-trained object detection models, specifically focusing on detecting faces in images and live video feeds. The goal of this project is to implement and compare different object detection models, starting with YOLO (You Only Look Once), and assess their performance in terms of accuracy and speed.

## Objectives:
1. Research and select a suitable pre-trained object detection model (e.g., YOLOv5 or YOLOv8).
2. Validate the model using a sample dataset to assess its performance.
3. Implement the model to detect faces in static images and output annotated images with bounding boxes.
4. Extend the functionality to handle live video feeds from a webcam, with real-time face detection and bounding box visualization.

Throughout the notebook, we will evaluate key metrics such as accuracy (mAP) and inference speed (FPS), making adjustments as needed to optimize for both performance and accuracy.


### Import necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
import os
import cv2
import zipfile
import random
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/student/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Create Folders

In [2]:
if not os.path.exists('Training'):
    os.makedirs('Training')

if not os.path.exists('Validation'):
    os.makedirs('Validation')

if not os.path.exists('Training/labels'):
    os.makedirs('Training/labels')

if not os.path.exists('Validation/labels'):
    os.makedirs('Validation/labels')

if not os.path.exists('Training/images'):
    os.makedirs('Training/images')

if not os.path.exists('Validation/images'):
    os.makedirs('Validation/images')



### Unzip data

In [3]:
def unzip_images_only(zip_file_path, dest_dir, images_location):
    """
    Unzips only the 'images' folder from a given zip file to the specified destination directory.
    It extracts all images from the 'WIDER_train/images' folder and places them directly into 'dest_dir/images'.

    Parameters:
    zip_file_path (str): Path to the zip file to be extracted.
    dest_dir (str): Directory where the images should be extracted, into a subdirectory named 'images'.
    images_location (str): The folder path inside the zip file where images are located.
    """
    # Define the full path for the 'images' subdirectory
    images_dest_dir = os.path.join(dest_dir, 'images')

    # Ensure the 'images' subdirectory exists
    if not os.path.exists(images_dest_dir):
        os.makedirs(images_dest_dir)

    # Open the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            # Extract only files from the specified 'images_location' folder
            if file_info.filename.startswith(images_location) and not file_info.is_dir():
                # Flatten the structure by moving all images directly into 'images_dest_dir'
                # Extract the filename only (ignore subdirectories)
                filename = os.path.basename(file_info.filename)
                target_path = os.path.join(images_dest_dir, filename)
                
                # Extract the file to the target path
                with zip_ref.open(file_info) as source, open(target_path, 'wb') as target:
                    target.write(source.read())

        print(f"Extracted all images from '{zip_file_path}' to '{images_dest_dir}' successfully, with flattened directory structure.")


# Unzip train data
zip_file_path_train = 'WIDER_train.zip'
dest_dir_train = 'Training'
images_location_train = 'WIDER_train/images/'

if not os.listdir('Training/images'):
    unzip_images_only(zip_file_path_train, dest_dir_train, images_location_train)

# Unzip validation data
zip_file_path_val = 'WIDER_val.zip'
dest_dir_val = 'Validation'
images_location_val = 'WIDER_val/images/'

if not os.listdir('Validation/images'):
    unzip_images_only(zip_file_path_val, dest_dir_val, images_location_val)

Extracted all images from 'WIDER_train.zip' to 'Training/images' successfully, with flattened directory structure.
Extracted all images from 'WIDER_val.zip' to 'Validation/images' successfully, with flattened directory structure.


### Unzip GT data

In [8]:

def unzip_file(zip_file_path, dest_dir="."):
    """
    Unzips a given zipped file to the specified destination directory.

    Parameters:
    zip_file_path (str): Path to the zip file to be extracted.
    dest_dir (str): Directory where the zip file should be extracted.
                    Default is the current working directory.
    """
    # Check if the destination directory exists, if not, create it
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Check if the zip file exists
    if not os.path.exists(zip_file_path):
        print(f"Error: Zip file '{zip_file_path}' does not exist.")
        return

    # Unzipping the file
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(dest_dir)
            print(f"Extracted '{zip_file_path}' to '{os.path.abspath(dest_dir)}' successfully.")
    except zipfile.BadZipFile:
        print(f"Error: '{zip_file_path}' is not a valid zip file.")

# Example usage
zip_file_path = 'wider_face_split.zip'
#dest_dir = './extracted_files'

unzip_file(zip_file_path)


Extracted 'wider_face_split.zip' to '/home/student/SSY340-Project/object-detection' successfully.


### Convert Annotations From the Wider Format to the YOLO Format

In [9]:

# Function to parse annotations and convert them to YOLO format
def convert_to_yolo_format(annotations_file, images_root, output_labels_dir):
    with open(annotations_file, 'r') as file:
        lines = file.readlines()

    i = 0
    while i < len(lines):
        # Parse image path
        image_path = lines[i].strip()

        # Skip if this line does not represent an image path
        if not image_path.endswith('.jpg'):
            i += 1
            continue

        # Extract only the filename (ignore any subdirectory in the annotation)
        image_filename = os.path.basename(image_path)
        image_full_path = os.path.join(images_root, image_filename)

        # Skip if image does not exist
        if not os.path.exists(image_full_path):
            print(f"Warning: Image {image_full_path} does not exist.")
            i += 1
            # Skip to the next image by finding the next valid image path
            while i < len(lines) and not lines[i].strip().endswith('.jpg'):
                i += 1
            continue

        # Load image to get dimensions
        image = cv2.imread(image_full_path)
        if image is None:
            print(f"Error: Failed to load image {image_full_path}")
            i += 1
            # Skip to the next image by finding the next valid image path
            while i < len(lines) and not lines[i].strip().endswith('.jpg'):
                i += 1
            continue

        image_height, image_width, _ = image.shape

        # Parse number of bounding boxes
        try:
            num_boxes = int(lines[i + 1].strip())
        except ValueError:
            print(f"Error: Expected an integer for number of bounding boxes but got: {lines[i + 1].strip()}")
            i += 1
            # Skip to the next image by finding the next valid image path
            while i < len(lines) and not lines[i].strip().endswith('.jpg'):
                i += 1
            continue

        # Create the label file path in the output labels directory
        label_path = os.path.join(output_labels_dir, image_filename.replace('.jpg', '.txt'))

        with open(label_path, 'w') as label_file:
            # Loop through bounding boxes
            for j in range(num_boxes):
                try:
                    box_data = list(map(int, lines[i + 2 + j].split()[:4]))  # Extract x, y, width, height
                    x, y, width, height = box_data

                    # Convert to YOLO format
                    x_center = (x + width / 2) / image_width
                    y_center = (y + height / 2) / image_height
                    width_normalized = width / image_width
                    height_normalized = height / image_height

                    # Write to label file in YOLO format
                    label_file.write(f"0 {x_center} {y_center} {width_normalized} {height_normalized}\n")
                except ValueError:
                    print(f"Warning: Failed to parse bounding box for {image_filename}, skipping this bounding box.")
                    continue

        # Move to the next image's annotations
        i += 2 + num_boxes

# Define paths

# Training Data
annotations_file_training = 'wider_face_split/wider_face_train_bbx_gt.txt'
images_root_training = 'Training/images'
output_labels_dir_training = 'Training/labels'

if not os.path.exists(output_labels_dir_training):
    os.makedirs(output_labels_dir_training)

# Validation Data
annotations_file_validation = 'wider_face_split/wider_face_val_bbx_gt.txt'
images_root_validation = 'Validation/images'
output_labels_dir_validation = 'Validation/labels'

if not os.path.exists(output_labels_dir_validation):
    os.makedirs(output_labels_dir_validation)

# Convert the annotations to YOLO format
if not os.listdir(output_labels_dir_training):
    convert_to_yolo_format(annotations_file_training, images_root_training, output_labels_dir_training)

if not os.listdir(output_labels_dir_validation):
    convert_to_yolo_format(annotations_file_validation, images_root_validation, output_labels_dir_validation)


### Load Pre-Trained YOLO v8n model

In [10]:
model = YOLO('yolov8n.pt')

### Verify model

In [11]:
# Define the path to the images directory
images_dir = 'Training/images/'

# List all image files in the directory (directly under images_dir)
image_files = [os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith('.jpg')]

# Ensure there are images available
if len(image_files) == 0:
    print("No images found in the directory.")
else:
    # Randomly select an image from the list
    random_image = random.choice(image_files)
    print(f"Randomly selected image: {random_image}")

    # Run inference on the selected image
    results = model(random_image)

    # Display the results
    results[0].show()

Randomly selected image: Training/images/48_Parachutist_Paratrooper_Parachutist_Paratrooper_48_660.jpg

image 1/1 /home/student/SSY340-Project/object-detection/Training/images/48_Parachutist_Paratrooper_Parachutist_Paratrooper_48_660.jpg: 480x640 9 persons, 1 laptop, 2 cell phones, 61.6ms
Speed: 3.3ms preprocess, 61.6ms inference, 86.2ms postprocess per image at shape (1, 3, 480, 640)


/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpsisa7ays.PNG'


### Train model

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.train(data='data.yaml', epochs=50, batch=16, imgsz=640)

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=Fal

100%|██████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 105MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 249 layers, 2,690,403 parameters, 2,690,387 gradients, 6.9 GFLOPs

Transferred 58/391 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train7', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...


100%|████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:00<00:00, 380MB/s]


...

      50/50      11.8G      1.344     0.6364     0.9598        111        640: 100%|██████████| 805/805 [02:08<00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/10


                   all       3225      39675      0.841      0.564      0.648      0.353

50 epochs completed in 2.170 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 5.6MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 5.6MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.15 🚀 Python-3.11.10 torch-2.3.1+cu118 CUDA:0 (Tesla T4, 14918MiB)
Model summary (fused): 186 layers, 2,684,563 parameters, 0 gradients, 6.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/10


                   all       3225      39675      0.836      0.566      0.648      0.353
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f5b2a8c1050>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [17]:
trained_model = YOLO('yolov8n.pt')

# Define the path to the images directory
images_dir = 'Training/images/'

# List all image files in the directory (directly under images_dir)
image_files = [os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith('.jpg')]

# Ensure there are images available
if len(image_files) == 0:
    print("No images found in the directory.")
else:
    # Randomly select an image from the list
    random_image = random.choice(image_files)
    print(f"Randomly selected image: {random_image}")

    # Run inference on the selected image
    results = trained_model(random_image)

    # Display the results
    results[0].show()

Randomly selected image: Training/images/13_Interview_Interview_Sequences_13_287.jpg

image 1/1 /home/student/SSY340-Project/object-detection/Training/images/13_Interview_Interview_Sequences_13_287.jpg: 224x640 7 persons, 85.0ms
Speed: 1.5ms preprocess, 85.0ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 640)


/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmp1dfks87a.PNG'
